# Config 클래스를 활용한 모드 전환

In [139]:
class Config:
    enable_backprop = True

In [140]:
import numpy as np

In [141]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [142]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록한다(부모 세대 + 1).

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 함수를 가져온다.

            # 수정 전: gys = [output.grad for output in f.outputs] # ❶
            gys = [output().grad for output in f.outputs] # ❶
            gxs = f.backward(*gys) # ❷
            if not isinstance(gxs, tuple): # ❸
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ❹
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    # +=을 쓰지 않는 이유: 값을 덮어쓰는게(인플레이스) 아니라 복사하게끔 + 사용
                    # 인플레이스 연산:  연산에 대한 결괏값을 새로운 변수에 저장하는 것이 아닌
                    # 기존 데이터를 대체하는 것을 의미
                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None # y는 약한 참조(weakref)

In [143]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)        # ❶ 별표를 붙여 언팩
        if not isinstance(ys, tuple): # ❷ 튜플이 아닌 경우 추가 지원
            ys = (ys,)

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs]) # 세대 설정
            for output in outputs:
                output.set_creator(self) # 출력 변수에 창조자를 설정한다.

            self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [144]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [145]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

def add(x0, x1):
    f = Add()
    return f(x0, x1)

# 필요 없는 미분 값 삭제

In [146]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


# 모드 전환

In [147]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))

In [148]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')    # 전처리
    try:
        yield
    finally:
        print('done') # 후처리

with config_test():
    print('process...')

start
process...
done


In [149]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [150]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [151]:
def no_grad():
    return using_config('enable_backprop', False)

In [152]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)